In [ ]:
# Google Driveをマウント（接続）する
from google.colab import drive
import os

# Google Driveに接続（初回は認証が必要です）
drive.mount('/content/drive')

# 画像を保存するディレクトリのパス
images_dir = "/content/drive/MyDrive/images"

# ディレクトリが存在しない場合は作成する（exist_ok=Trueで既存の場合はエラーにしない）
os.makedirs(images_dir, exist_ok=True)

# 作成完了メッセージを表示
print("✅ フォルダ作成:", images_dir)

## ステップ1: Google Driveの接続とフォルダ作成

Google Driveに接続して、編集したい画像を保存するフォルダを作成します。

## ステップ2: 必要なライブラリのインストール

画像編集に必要なライブラリをインストールします:
- **diffusers**: Stable Diffusionベースのモデル用
- **accelerate**: モデルの高速化
- **safetensors**: モデルの安全な読み込み
- **transformers**: テキスト処理用

## ステップ3: Instruct-Pix2Pixモデルの読み込み

**timbrooks/instruct-pix2pix**モデルを読み込みます。このモデルは、テキスト指示に基づいて画像を編集できます。

### モデル設定
- **torch_dtype=float16**: メモリ効率化のため16bit精度を使用
- **safety_checker=None**: セーフティチェックを無効化
- **EulerAncestralDiscreteScheduler**: 高品質な画像生成用スケジューラー

## ステップ4: サンプル画像で編集を試す

まず、インターネット上のサンプル画像を使って、画像編集を試してみます。

### 編集プロンプトの例
- "turn him into cyborg"（サイボーグに変える）
- "make it winter"（冬の風景にする）
- "turn into a painting"（絵画風にする）
- "add sunglasses"（サングラスを追加）

### パラメータの意味
- **num_inference_steps**: 生成ステップ数（多いほど高品質だが時間がかかる）
- **image_guidance_scale**: 元画像への忠実度（大きいほど元画像に近い）
- **guidance_scale**: プロンプトへの忠実度（大きいほどプロンプトに忠実）

## ステップ5: 自分の画像で編集を試す

Google Driveにアップロードした自分の画像を使って編集してみましょう。

### 使い方
1. Google Driveの`/MyDrive/images/`フォルダに画像をアップロード
2. `prompt`変数に編集内容を英語で記述
3. パラメータを調整（必要に応じて）
4. セルを実行

### プロンプトのコツ
- **具体的に**: "make it dark"より"make the background dark and ominous"の方が良い
- **簡潔に**: 長すぎるプロンプトは避ける
- **英語で**: 日本語より英語の方が精度が高い

### パラメータ調整のヒント
- **元画像を保ちたい**: `image_guidance_scale`を大きく（1.5〜2.0）
- **大胆に変更したい**: `image_guidance_scale`を小さく（0.5〜1.0）
- **プロンプトに忠実に**: `guidance_scale`を大きく（8.0〜10.0）

# 深層学習体験会: AI画像編集（Instruct-Pix2Pix）

このノートブックでは、**Instruct-Pix2Pix**モデルを使って、テキスト指示で画像を編集する方法を学びます。

## Instruct-Pix2Pixとは？

**Instruct-Pix2Pix**は、自然言語の指示に従って画像を編集するAIモデルです。「サイボーグに変えて」「背景を暗くして」などの指示を与えるだけで、自動的に画像を編集できます。

### 主な機能
- **テキストベースの編集**: 自然言語で編集内容を指示
- **柔軟な変換**: スタイル変更、オブジェクト追加、色調補正など
- **元画像の保持**: 基本的な構図を保ちながら編集

### 技術的背景
Stable Diffusionをベースに、画像編集タスク用にファインチューニングされたモデルです。

### 応用例
- 写真のスタイル変更（アニメ風、絵画風など）
- オブジェクトの追加・削除
- 天候や時間帯の変更
- ファッションアイテムの試着シミュレーション
- 建築物のデザイン変更

## このノートブックの流れ
1. Google Driveの接続
2. 必要なライブラリのインストール
3. モデルの読み込み
4. サンプル画像で編集を試す
5. 自分の画像で編集を試す

**注意**: このノートブックはT4 GPUの使用を推奨します。

In [ ]:
!pip install diffusers accelerate safetensors transformers

In [ ]:
# 必要なライブラリをインポート
import PIL  # 画像処理用
import requests  # URLから画像をダウンロード
import torch  # PyTorchフレームワーク
from diffusers import StableDiffusionInstructPix2PixPipeline, EulerAncestralDiscreteScheduler

# 使用するモデルのID
model_id = "timbrooks/instruct-pix2pix"

# Instruct-Pix2Pixパイプラインを読み込む
# torch_dtype=torch.float16: メモリ節約のため16bit精度を使用
# safety_checker=None: セーフティチェックを無効化
pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(
    model_id, 
    torch_dtype=torch.float16, 
    safety_checker=None
)

# モデルをGPUに転送（CUDAを使用）
pipe.to("cuda")

# スケジューラーをEulerAncestralDiscreteに設定（高品質な画像生成用）
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

In [ ]:
# 必要なライブラリをインポート
import requests, PIL
from PIL import Image, ImageOps
import matplotlib.pyplot as plt

# --- サンプル画像をダウンロード ---
url = "https://raw.githubusercontent.com/timothybrooks/instruct-pix2pix/main/imgs/example.jpg"

def download_image(path_or_url):
    """
    URLまたはローカルパスから画像を読み込む関数
    
    引数:
        path_or_url: 画像のURLまたはローカルパス
    戻り値:
        PIL.Image: RGB形式の画像
    """
    if isinstance(path_or_url, str) and path_or_url.startswith(("http://", "https://")):
        # URLの場合: インターネットから画像をダウンロード
        image = Image.open(requests.get(path_or_url, stream=True).raw)
    elif os.path.exists(path_or_url):
        # ローカルパスの場合: ファイルから画像を開く
        image = Image.open(path_or_url)
    else:
        # どちらでもない場合: エラーを発生させる
        raise ValueError(f"無効なパスまたはURL: {path_or_url}")

    # EXIFデータに基づいて画像の向きを修正
    image = ImageOps.exif_transpose(image)
    # RGB形式（カラー）に変換して返す
    return image.convert("RGB")

# サンプル画像をダウンロード
image = download_image(url)

# --- 編集プロンプトを設定 ---
# "彼をサイボーグに変えて"という指示
prompt = "turn him into cyborg"

# --- 画像編集の実行 ---
result = pipe(
    prompt=prompt,  # 編集指示
    image=image,  # 入力画像
    num_inference_steps=10,  # 生成ステップ数（多いほど高品質だが時間がかかる）
    image_guidance_scale=1.0,  # 元画像への忠実度
    guidance_scale=7.5,  # プロンプトへの忠実度
).images[0]  # 結果の最初の画像を取得

# --- 結果の表示 ---
# 1行2列のサブプロット（2つの画像を横に並べる）
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# 左側: 元画像
ax[0].imshow(image)
ax[0].set_title("元画像")
ax[0].axis("off")

# 右側: 編集後の画像
ax[1].imshow(result)
ax[1].set_title(f"編集後の画像\nプロンプト: '{prompt}'")
ax[1].axis("off")

# レイアウトを調整して表示
plt.tight_layout()
plt.show()

In [ ]:
# --- 自分の画像で画像編集を試す ---

# Google Driveから画像を読み込む
image = download_image("/content/drive/MyDrive/images/07_01.png")

# 編集プロンプト: "背景を暗く不気味にする"
prompt = "make the background dark and ominous"

# 画像編集を実行
result = pipe(
    prompt=prompt,  # 編集指示
    image=image,  # 入力画像
    num_inference_steps=15,  # 生成ステップ数（前回より多く設定）
    guidance_scale=7.5,  # プロンプトへの忠実度
    image_guidance_scale=1.2,  # 元画像への忠実度（少し高めに設定）
).images[0]

# --- 結果の表示 ---
fig, ax = plt.subplots(1,2, figsize=(12,6))

# 左側: 元画像
ax[0].imshow(image)
ax[0].set_title("元画像")
ax[0].axis("off")

# 右側: 編集後の画像
ax[1].imshow(result)
ax[1].set_title(f"編集後\n'{prompt}'")
ax[1].axis("off")

# レイアウトを調整して表示
plt.tight_layout()
plt.show()